In [1]:
import os
from src.Transformation.common.kw_extractors import YAKEKeywordExtractor
from src.Transformation.common.text_clustering import LazyFuzzyClassifier
from src.Transformation.common.spark_udfs import create_keyword_extractor_udf, create_clean_country_udf, create_text_classefier_udf
from global_variables import LANDING_DIR, LUKES_DATASET_NAME, CLEAN_LOCATION_DATASET_NAME

gsearch_file_path = os.path.join(LANDING_DIR, LUKES_DATASET_NAME)
countries_file_path = os.path.join(LANDING_DIR, CLEAN_LOCATION_DATASET_NAME)

txt_classefier = LazyFuzzyClassifier()
kw_extractor = YAKEKeywordExtractor()

import pandas as pd
countries_df = pd.read_csv(countries_file_path)


iso2 = countries_df["iso2"].dropna().astype(str).tolist()
iso3 = countries_df["iso3"].dropna().astype(str).tolist()
country_names = countries_df["country"].dropna().astype(str).tolist()

del countries_df

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list, col, flatten, size, explode, desc, concat, lit

In [3]:
spark = SparkSession.builder.appName("kw_extractor").getOrCreate()

24/12/02 21:05:19 WARN Utils: Your hostname, ahmad-HP-ZBook-17 resolves to a loopback address: 127.0.1.1; using 192.168.1.72 instead (on interface wlo1)
24/12/02 21:05:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 21:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
kw_extractor_bc         = spark.sparkContext.broadcast(kw_extractor)
txt_classefier_bc       = spark.sparkContext.broadcast(txt_classefier)
iso2_bc                 = spark.sparkContext.broadcast(iso2)
iso3_bc                 = spark.sparkContext.broadcast(iso3)
country_names_bc        = spark.sparkContext.broadcast(country_names)
candidate_lables_bc     = spark.sparkContext.broadcast(["Data Engineer", "Data Analyst", "Business Analyst", "BI Developer", "Data Scientist"])

keyword_extractor_udf   = create_keyword_extractor_udf(kw_extractor_bc)    
text_classefier_udf     = create_text_classefier_udf(txt_classefier_bc, candidate_lables_bc)
clean_country_udf       = create_clean_country_udf(iso2_list_bc=iso2_bc, iso3_list_bc=iso3_bc, country_names_list_bc=country_names_bc)

In [5]:
df = spark.read.option("header",True).csv(gsearch_file_path)

In [6]:
enriched = (df.select(col("title"), col("location"), col("description")).filter(
        col("location").isNotNull() & col("description").isNotNull()
    )
    .limit(10000)
    .withColumn("country", clean_country_udf(col("location")))
    .filter(col("country").isNotNull())
    .withColumn("kw_arr", keyword_extractor_udf(col("description")))
    .filter(size(col("kw_arr")) > 0)
    .withColumn("clean_title", text_classefier_udf(col("title")))
    .filter(col("clean_title").isNotNull())
    .select(col("country"), col("clean_title"), col("kw_arr"))
    .groupBy(col("clean_title"), col("country"))
    .agg(collect_list(col("kw_arr")).alias("kw_arr"))
    .withColumn("keywords", flatten(col("kw_arr")))
    .drop("kw_arr")
)

enriched.show(truncate=True)

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']


+----------------+------------------+--------------------+
|     clean_title|           country|            keywords|
+----------------+------------------+--------------------+
|   Data Engineer|          Anywhere|[{Engineering Dat...|
|Business Analyst|             Italy|[{effective commu...|
|    BI Developer|     United states|[{Job Title, 0.01...|
|    Data Analyst|           Iceland|[{data transforma...|
|    BI Developer|           Burundi|[{Microsoft appli...|
|Business Analyst|           Austria|[{members, 0.1583...|
|    Data Analyst|           Andorra|[{paid time, 0.09...|
|    Data Analyst|     United states|[{Overview, 0.044...|
|    Data Analyst|            Canada|[{release the ful...|
|  Data Scientist|           Andorra|[{Paid Time, 0.03...|
|   Data Engineer|              Peru|[{Critical Illnes...|
|Business Analyst|        Montserrat|[{supporting or w...|
|Business Analyst|Dominican republic|[{segments includ...|
|    BI Developer|        Guadeloupe|[{Excel, 0.044911..

In [7]:
from pyspark.sql.functions import col, explode, sum, desc, count
from pyspark.sql.types import DoubleType

exploded = (
    enriched
    # .filter(col("clean_title").like("%ngineer%"))
    .withColumn("exploded", explode("keywords"))
    .select(
        col("country"), 
        col("clean_title"),
        col("exploded.keyword").alias("keyword"),
        col("exploded.weight").cast(DoubleType()).alias("weight")
    )
    .groupBy(col("country"), col("clean_title"), col("keyword"))
    .agg(
        sum("weight").alias("total_weight"),
        count("keyword").alias("keyword_cnt")
    )
    .withColumn("score", col("total_weight") * col("keyword_cnt")) 
    .orderBy(desc("score"))
)


In [9]:
# Install prettytable
# ! pip install prettytable

from prettytable import PrettyTable

# Create a PrettyTable object
table = PrettyTable()

df_filtered = (
    exploded
    .filter(col("clean_title") == "Data Engineer")
    .filter(col("country") == "United states" & col("country") == "Anywhere")
    .limit(100)
)

# Set the columns
table.field_names = exploded.columns

# Add rows to the table
for row in exploded.collect():
    table.add_row(row)

# Print the table in a pretty format
print(table)

Py4JError: An error occurred while calling o195.and. Trace:
py4j.Py4JException: Method and([class java.lang.String]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:321)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:329)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)

